In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#data-exploration

df=pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')

In [ ]:
df.head()

In [ ]:
sns.set_style('darkgrid')
sns.countplot(df.price_range)

# Inference : It's a multiclass-class classification problem
### the data is unformly distributed among all classes
### Cross Validation - Kfold would be fine no need of stratified
### Evaluation metric - macro-F1 score or accuracy would be fine seeing the uniform distribution

In [ ]:
#create_folds.py
from sklearn import model_selection

df['kfold']=-1

df=df.sample(frac=1).reset_index(drop=False)

kf=model_selection.KFold(n_splits=5)

for fold,(trn_,val_) in enumerate(kf.split(X=df)):
    df.loc[val_,'kfold']=fold


In [ ]:
df.drop('index',axis=1,inplace=True
       )

In [ ]:
df.head()

In [ ]:
#rf-classifier-train.py

from sklearn import ensemble
from sklearn import metrics


def run(fold):
    
    #train-val-split
    
    df_train=df[df.kfold!=fold].reset_index(drop=True)
    df_val=df[df.kfold==fold].reset_index(drop=True)

    x_train=df_train.drop('price_range',axis=1).values
    y_train=df_train['price_range']

    x_val=df_val.drop('price_range',axis=1).values
    y_val=df_val['price_range']


    clf=ensemble.RandomForestClassifier(criterion='entropy',max_depth=123,n_estimators=225)
    clf.fit(x_train,y_train)

    preds=clf.predict(x_val)

    accuracy=metrics.accuracy_score(y_val,preds)
    
    print(f'fold : {fold} , Accuracy = {accuracy}')
    
if __name__=='__main__':
    for f_ in range(5):
        run(f_)

    Inference : without any hyperparameter-optimization or feature engineering the model seems pretty accurate
    
   <b> Best Accuracy for Random-Forest = 88.75 %

# hyperparameter optimization

 {
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
)

In [ ]:
param_grid = {'n_estimators':[100,200,250,300,400,500],'criterion':['gini','entropy'],
              'max_depth':[1,2,5,7,11,15],
}

In [ ]:
from sklearn import metrics,model_selection


classifier=ensemble.RandomForestClassifier(n_jobs=-1)

param_grid = {'n_estimators':[100,200,250,300,400,500],'criterion':['gini','entropy'],
              'max_depth':[1,2,5,7,11,15],
}

#initalzing the grid search 
#estimator below is the model that we have defined
#accuracy is defined as the metric
#cv=5 means we are using 5 fold cv(cross-validation) (not straified)

model=model_selection.GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    
    scoring='accuracy',
    verbose=10,
    n_jobs=-1,
    cv=5

)

x_train=df[df.columns].drop('price_range',axis=1)
y_train=df.price_range.values
model.fit(x_train,y_train)

print(f'Best Score = {model.best_score_}')

print('Best parameter set : ')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
     print(f"\t{param_name}: {best_parameters[param_name]}")

Was able to improve about 1 % 

# lol that wasn't of much help
let's look at the data again !

# Back to exploration and engineering

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
#checking highly correlated features
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df, 3))

Inference : No highly releated pairs :(

In [ ]:
#let's try KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def run(fold):

    df_train=df[df.kfold!=fold].reset_index(drop=True)
    df_val=df[df.kfold==fold].reset_index(drop=True)

    x_train=df_train.drop('price_range',axis=1).values
    y_train=df_train['price_range']

    x_val=df_val.drop('price_range',axis=1).values
    y_val=df_val['price_range']


    
    clf=KNeighborsClassifier(n_neighbors=15)
    clf.fit(x_train,y_train)

    preds=clf.predict(x_val)

    accuracy=metrics.accuracy_score(y_val,preds)
    
    print(f'fold : {fold} , Accuracy = {accuracy}')
    
if __name__=='__main__':
    for f_ in range(5):
        run(f_)

In [ ]:
#Nice without any hyperparameter tuning ...it's already better than RandomForest

In [ ]:
#trying to find the perfect k-value

In [ ]:
fold=0
df_train=df[df.kfold!=fold].reset_index(drop=True)
df_val=df[df.kfold==fold].reset_index(drop=True)

x_train=df_train.drop('price_range',axis=1).values
y_train=df_train['price_range']

x_val=df_val.drop('price_range',axis=1).values
y_val=df_val['price_range']

acc=[]
for k in range(1,30):
    clf=KNeighborsClassifier(n_neighbors=k)
    clf.fit(x_train,y_train)
    preds=clf.predict(x_val)

    accuracy=metrics.accuracy_score(y_val,preds)
    acc.append(accuracy)

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,5))
plt.xlabel('K-neighbour')
plt.ylabel('accuracy')
sns.set_style('darkgrid')
sns.lineplot(x=list(range(1,30)),y=acc,color='red')

Inference KNN performs way better than,without much tuning